In [33]:
from sklearn import datasets
import pandas as pd
import sqlite3
import tensorflow as tf
import numpy as np

# Datasets

In [3]:
wine = datasets.load_wine()
wine.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [5]:
x_data = wine['data']
x_data.shape

(178, 13)

In [35]:
y_data = wine['target']
y_data

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

# DataFrame

In [ ]:
df_wine_x = pd.DataFrame(wine['data'])
df_wine_x.info()

In [ ]:
df_wine_y = pd.DataFrame(wine['target'])
df_wine_y.info()

# DB

In [49]:
connect = sqlite3.connect('./db.sqlite3')
df_wine_x.to_sql('wine_x_data', connect, if_exists='append', index=False)
df_wine_y.to_sql('wine_y_data', connect, if_exists='append', index=False)

In [67]:
x_load = pd.read_sql_query('select * from wine_x_data', connect)
y_load = pd.read_sql_query('select * from wine_y_data', connect)
x_load.head(4), y_load.head(4)

(       0     1     2     3      4     5  ...     7     8     9    10    11      12
 0  14.23  1.71  2.43  15.6  127.0  2.80  ...  0.28  2.29  5.64  1.04  3.92  1065.0
 1  13.20  1.78  2.14  11.2  100.0  2.65  ...  0.26  1.28  4.38  1.05  3.40  1050.0
 2  13.16  2.36  2.67  18.6  101.0  2.80  ...  0.30  2.81  5.68  1.03  3.17  1185.0
 3  14.37  1.95  2.50  16.8  113.0  3.85  ...  0.24  2.18  7.80  0.86  3.45  1480.0
 
 [4 rows x 13 columns],    0
 0  0
 1  0
 2  0
 3  0)

In [68]:
x_data = x_load.to_numpy()
y_data = y_load.to_numpy()

# Model

In [69]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=(13,)))

model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))

model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

In [77]:
model.fit(x_load, y_load, epochs=500, validation_split=0.3)

Epoch 1/500
4/4 [==============================] - 0s 33ms/step - loss: 0.2216 - acc: 0.9355 - val_loss: 92.7656 - val_acc: 0.1111
Epoch 2/500
4/4 [==============================] - 0s 9ms/step - loss: 0.1381 - acc: 0.9435 - val_loss: 94.0877 - val_acc: 0.1111
Epoch 3/500
4/4 [==============================] - 0s 10ms/step - loss: 0.2238 - acc: 0.9194 - val_loss: 91.9185 - val_acc: 0.1111
Epoch 4/500
4/4 [==============================] - 0s 9ms/step - loss: 0.2648 - acc: 0.9113 - val_loss: 95.3203 - val_acc: 0.1111
Epoch 5/500
4/4 [==============================] - 0s 9ms/step - loss: 0.2377 - acc: 0.9113 - val_loss: 91.6416 - val_acc: 0.1111
Epoch 6/500
4/4 [==============================] - 0s 8ms/step - loss: 0.1964 - acc: 0.9113 - val_loss: 94.2025 - val_acc: 0.1111
Epoch 7/500
4/4 [==============================] - 0s 9ms/step - loss: 0.2278 - acc: 0.9194 - val_loss: 91.7063 - val_acc: 0.1111
Epoch 8/500
4/4 [==============================] - 0s 8ms/step - loss: 0.2051 - acc: 0.9

# Evaluation

In [78]:
model.evaluate(x_load, y_load)

6/6 [==============================] - 0s 2ms/step - loss: 18.0856 - acc: 0.6798


[18.085617065429688, 0.6797752976417542]

# Predict

In [82]:
x_data[17], y_data[17]

(array([1.383e+01, 1.570e+00, 2.620e+00, 2.000e+01, 1.150e+02, 2.950e+00,
        3.400e+00, 4.000e-01, 1.720e+00, 6.600e+00, 1.130e+00, 2.570e+00,
        1.130e+03]), array([0]))

In [83]:
pred = model.predict([[1.383e+01, 1.570e+00, 2.620e+00, 2.000e+01, 1.150e+02, 2.950e+00,
        3.400e+00, 4.000e-01, 1.720e+00, 6.600e+00, 1.130e+00, 2.570e+00,
        1.130e+03]])
pred

array([[0.9510792 , 0.04892075, 0.        ]], dtype=float32)

In [84]:
np.argmax(pred)

0